This is a catboost baseline for prediction time to eruption using the generated dataset with TSFresh library. 
I used the EfficientFCParameters() parameter that results in 773 features generation for each time series. Both train and test datasets were processing during 5 days on 30 threads of Ryzen 1950X. Finally, we have 7730 features for each observation. The dataset is uploaded as ingv-tsfresh-7730. Let's improve the scores :) 

In [ ]:
import os
import gc
import numpy as np
import pandas as pd

from time import time
from time import ctime

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
from tqdm import tqdm

import joblib
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()-1

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

def plotfig (ypred, yactual, strtitle, y_max):
    plt.scatter(ypred, yactual.values.ravel())
    plt.title(strtitle)
    plt.plot([(0, 0), (y_max, y_max)], [(0, 0), (y_max, y_max)])
    plt.xlim(0, y_max)
    plt.ylim(0, y_max)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.show()

### Load prepared dataset

In [ ]:
train = pd.read_csv('../input/ingv-tsfresh-7730/train.csv', sep = ';')
train.set_index('Unnamed: 0', inplace = True)
test = pd.read_csv('../input/ingv-tsfresh-7730/test.csv', sep = ';')
test.set_index('Unnamed: 0', inplace = True)

### Or create it using the TSFresh lib

In [ ]:
%%time

Y = train['time_to_eruption']
X = train.drop(['time_to_eruption'], axis = 1)
X_test = test

n_fold = 5
cv = KFold(n_splits=n_fold, shuffle=True, random_state=42)

oof = np.zeros(len(X))
cat_prediction = np.zeros(len(X_test))
mae, r2 = [], []

PARAMS = {
    
             'random_seed': 42,
             'eval_metric': 'MAE',
#              'iterations': 100,
             'task_type': 'GPU',

        }

for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    print('\nFold', fold_n, 'started at', ctime())

    X_train = X.iloc[train_index,:]
    X_valid = X.iloc[valid_index,:]
    
    Y_train = Y.iloc[train_index]
    Y_valid = Y.iloc[valid_index]
          
    best_model = CatBoostRegressor(**PARAMS, thread_count = -1)  
    
    train_dataset = Pool(data=X_train,
                     label=Y_train,
                     )
    
    eval_dataset = Pool(data=X_valid,
                    label=Y_valid,
                    )
    
    best_model.fit(train_dataset,
              use_best_model=True,
              verbose = False,
              plot = True,
              eval_set=eval_dataset,
              early_stopping_rounds=100)

   
    y_pred = best_model.predict(Pool(data=X_valid))

    mae.append(mean_absolute_error(Y_valid, y_pred))
    r2.append(r2_score(Y_valid, y_pred))

    print('MAE: ', mean_absolute_error(Y_valid, y_pred))
    print('R2: ', r2_score(Y_valid, y_pred))

    cat_prediction += best_model.predict(Pool(data=X_test))
        
cat_prediction /= n_fold

print('='*45)
print('CV mean MAE: {0:.4f}, std: {1:.4f}.'.format(np.mean(mae), np.std(mae)))
print('CV mean R2:  {0:.4f}, std: {1:.4f}.'.format(np.mean(r2), np.std(r2)))

In [ ]:
plotfig(best_model.predict(X), Y, 'Predicted vs. Actual responses for Catboost', max(Y) + 0.1*max(Y))

In [ ]:
submission = pd.DataFrame()
submission['segment_id'] = test.index
submission['time_to_eruption'] = cat_prediction
submission.to_csv('submission.csv', header=True, index=False)